In [12]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import math

In [13]:
train_data  = pd.read_csv("./Alphabets/train.csv",header=None).to_numpy()
test_data = pd.read_csv("./Alphabets/test.csv",header=None).to_numpy()

In [71]:
train_x = train_data[:,:-1]/255
train_y = train_data[:,-1]
test_x = test_data[:,:-1]/255
test_y = test_data[:,-1]

# Neural Network Class

In [225]:
"""Training algorithms for deep learning models are usually iterative in nature and thus 
require the user to specify some initial point from which to begin the iterations. 
Moreover, training deep models is a sufficiently difficult task that most algorithms are
strongly affected by the choice of initialization."""

class neuralNetwork:
    def __init__(self,batchSize,input_features,architecture,target_class,eta,max_iter,activationMode,learningRate):
        self.batchSize = batchSize
        self.input_features = input_features
        self.architecture = architecture
        self.target_class = target_class
        self.learningRate = learningRate
        self.activationMode  = activationMode

        self.parameter = []
        self.layer_input = [0]*(len(architecture)+1)
        self.layer_output = [0]*(len(architecture)+1)
        self.layer_delta = [0]*(len(architecture)+1)
        self.total_layers = len(architecture)+1

        self.eta = eta
        self.max_iter = max_iter

        

    def activation(self,x,mode):
        if mode=="sigmoid":
            return 1/(1+np.exp(-x))
        elif mode=="relu":
            return np.where(x<0,0,x)
        
        
        
    def differentiation(self,op,mode):
        if mode=="sigmoid":
            return op*(1-op)
        elif mode=="relu":
            return 1 * (op > 0)
        
        
        
    def oneHotEncoding(self,y): #y is **list** of ouput label 0<=y<26
        """ converts y label to vector representation called One Hot Encoding """
        a = np.array(y)
        b = np.zeros((a.size, self.target_class))
        b[np.arange(a.size),a] = 1
        return b

    

    def initalize_parameters(self):

        total_layers_architecture = self.architecture  + [self.target_class]
        layerInputSize = self.input_features
        np.random.seed(0)
        
        """Random intialization is used to preserve the stochastic nature of neural networks"""
        
        for layer, total_neurons in enumerate(total_layers_architecture):
            np.random.seed(layer)
            if layer ==0:
                neurons_in_prev = self.input_features
            else:
                neurons_in_prev = total_layers_architecture[layer-1]
                
            """HE(et.al 2015) INITIALIZATION IS USED TO REMOVE SYMMETRY ANS VANISHING GRADIENT ISSUES"""  
            
            layerOutputSize = total_neurons
            layer_weight = (np.random.randn(layerOutputSize, layerInputSize)*2)/math.sqrt(neurons_in_prev)
            """It is important to note that the bias weight in each neuron
            is set to zero by default, not a small random value."""
            layer_bias  = np.zeros((layerOutputSize,1))
            
            self.parameter.append([layer_weight,layer_bias])

            layerInputSize = layerOutputSize

        


    def full_feedForward(self,X):
        current_input = X.copy()

        for layer in range(self.total_layers):

            current_parameter = self.parameter[layer]

            weight = current_parameter[0]
            bias = current_parameter[1]
            netJ = np.dot(current_input,weight.T) + bias.T
            
            if self.activationMode == "relu":
                if layer == self.total_layers-1:
                    G_netJ  = self.activation(netJ,"sigmoid")   # only output layer 
                else :
                    G_netJ  = self.activation(netJ,"relu")   # all hidden layer
            
            elif self.activationMode == "sigmoid":
                G_netJ  = self.activation(netJ,"sigmoid") # all layers 
                    

            self.layer_output[layer] = G_netJ
            self.layer_input[layer] = current_input

            current_input = G_netJ.copy()
        """         last single layer output is the output of entire neural network
                     to be used for calculating new loss function value                   """




    def full_backpropagation(self,Y):

        """  start with last (ouyput)layer whose deltaJ is calculated
             differently then rest of hidden layer                    """
        lastlayer = self.total_layers-1
        op = self.layer_output[lastlayer]
        
        diff_op = self.differentiation(op,"sigmoid")  # in every case output layer is sigmoid
        
        deltaJ_lastlayer = (Y-op)*diff_op/(Y.shape[0])
        self.layer_delta[lastlayer] = deltaJ_lastlayer

        deltaJ_prev = deltaJ_lastlayer.copy()

        #reverse iteration
        for layer in range(self.total_layers-1,0,-1):
            theta_downNBR  = self.parameter[layer][0] # weight without bias

            oj = self.layer_output[layer-1]
            
            if self.activationMode =="relu":
                diff_oj = self.differentiation(oj,"relu")
            elif self.activationMode=="sigmoid":
                diff_oj = self.differentiation(oj,"sigmoid")
                
            deltaJ_curr = np.dot(deltaJ_prev, theta_downNBR)*diff_oj

            self.layer_delta[layer-1] = deltaJ_curr
            deltaJ_prev = deltaJ_curr.copy()
            
            

    def costFunction(self,y):
        final_op = self.layer_output[self.total_layers-1]
        return (np.sum((y-final_op)**2))/(2*y.shape[0])



    def updateParameters(self,epochCount):
        ETA = self.eta   #defalut for fault tolerance
        
        if self.learningRate == "normal":
            ETA = self.eta
        elif self.learningRate ==  "adaptive":
            ETA = self.eta/math.sqrt(epochCount) # as per question requirement 
            
        for i in range(len(self.architecture)+1):
            
            gradient_W = np.dot(self.layer_delta[i].T, self.layer_input[i])
            gradient_B = np.sum(self.layer_delta[i],axis = 0).T.reshape((-1,1))
            self.parameter[i][0] = self.parameter[i][0] + (ETA)*gradient_W
            self.parameter[i][1] = self.parameter[i][1] + (ETA)*gradient_B

            
            

    def fit(self,x,y):
        
        self.initalize_parameters()
        
        indexes = np.arange(x.shape[0])
        Y = self.oneHotEncoding(y)
        
        cost, newcost =0, 0
        improvement, old_improvement = math.inf , 0
        n_iter_no_change = 20
        i = 0
        epochNumber = 0
        totalBatches = math.ceil(x.shape[0]/self.batchSize)
        while(True):
            
            i+=1

            """shuffle the data after every_epoch to maintain stochastic nature(random) of the newtork"""
            np.random.shuffle(indexes)

            epochNumber = i
            
            for j in range(0,x.shape[0],self.batchSize):
                
                batch = indexes[j:j + self.batchSize]
                
                x_batch = x[batch]
                y_batch = Y[batch]

                self.full_feedForward(x_batch)
               
                self.full_backpropagation(y_batch)

                self.updateParameters(epochNumber)
                
                cost += self.costFunction(y_batch)

            oldcost = newcost
            newcost = cost/totalBatches
            cost = 0
            
            old_improvement = improvement
            improvement  = abs(oldcost - newcost)
            
            if improvement <= 1e-6:
                print("convergence reached with total epoch :",i)
                return i
                break
            
            if abs(improvement - old_improvement) < 1e-6:
                n_iter_no_change -= 1
            
            #early stopping
            if n_iter_no_change ==0:
                print("Converged after cost not improved for n_iter_no_change ",i)
                break
                
            if i == self.max_iter:
                print("max_iter reached")
                return i
                break   
                
            if i%100 ==0:
                print(improvement)
                print('Current Epoch is : ',i)
                
                

    def score(self,x,y):
        self.full_feedForward(x)
        final_op = self.layer_output[self.total_layers-1]
        return np.count_nonzero((np.argmax(final_op,axis =1) == y) == True)/y.shape[0] #,np.argmax(final_op,axis =1)


In [242]:
# batchSize| input_features| architecture| target_class| eta,max_iter| activationMode| learningRate
model = neuralNetwork(100,784,[5],26,0.5,3000,"sigmoid","normal")

In [ ]:
s = time.time()
model.fit(train_x,train_y)
print(time.time()-s)

0.000316585770587241
Current Epoch is :  100
0.0004337900719039167
Current Epoch is :  200
0.0002100065541764673
Current Epoch is :  300
0.00016918404887705796
Current Epoch is :  400
7.345551970266984e-05
Current Epoch is :  500
1.4370972459765152e-05
Current Epoch is :  600


In [240]:
model.score(train_x,train_y)

0.08838461538461538

In [241]:
model.score(test_x,test_y)

0.08846153846153847

# PART E

In [131]:
from sklearn.neural_network import MLPClassifier

In [132]:
def oneHotEncoding(y,targetClass): #y is **list** of ouput label 0<=y<26
    """ converts y label to vector representation called One Hot Encoding """
    a = np.array(y)
    b = np.zeros((a.size,targetClass ))
    b[np.arange(a.size),a] = 1
    return b

In [133]:
Y = oneHotEncoding(train_y,26)

In [134]:
# when i thought we need to train 26 models for each class as binary classification 

# models = []
# for i in tqdm(range(26)):
#     model = MLPClassifier(hidden_layer_sizes=(100,100,), activation='relu', solver='lbfgs', 
#                       alpha=0.0001, batch_size=100, learning_rate='adaptive', 
#                       learning_rate_init=0.5, power_t=0.5, max_iter=200, 
#                       shuffle=True, random_state=None, tol=0.0001, verbose=False, 
#                       warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
#                       validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
#                       n_iter_no_change=10, max_fun=15000)
#     model.fit(train_x,Y[:,i])
#     models.append(model)
    
# predictions_prob_train = []
# predictions_prob_test = []
# for i in range(26):
#     model  = models[i]
#     predictions_prob_train.append(model.predict_proba(train_x))
#     predictions_prob_test.append(model.predict_proba(test_x))

    
# def prediction(x,predictions_prob)
#     prediction= []
#     for j in range(x.shape[0]):
#         prob = 0
#         Class = -1
#         for i in range(26):

#             if prob < predictions_prob[i][j][1]:
#                 Class = i
#                 prob = predictions_prob[i][j][1]
#         prediction.append(Class)
#     return prediction


# from sklearn.metrics import accuracy_score

# accuracy_score(test_y,prediction(test_x,predictions_prob_test))*100
# accuracy_score(train_y,prediction(train_x,predictions_prob_train))*100

In [154]:
#making aplha =0 as we need to compare with our results which include no regularizer
model = MLPClassifier(hidden_layer_sizes=(100,100), activation='relu', solver='sgd', 
                      alpha=0, batch_size=100, learning_rate='invscaling', 
                      learning_rate_init=0.1, power_t=0.5, max_iter=2000, 
                      shuffle=True, random_state=None, tol=0.0001, verbose=False, 
                      warm_start=False, momentum=0.9, nesterovs_momentum=True, n_iter_no_change=10)
model.fit(train_x,Y)

MLPClassifier(activation='relu', alpha=0, batch_size=100, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='invscaling',
              learning_rate_init=0.1, max_fun=15000, max_iter=2000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [161]:
prob_train = model.predict_proba(train_x)
prob_test = model.predict_proba(test_x)

In [162]:
from sklearn.metrics import accuracy_score
print(accuracy_score(train_y,prob_train.argmax(axis = 1))*100)
print(accuracy_score(test_y,prob_test.argmax(axis = 1))*100)

91.35384615384615
87.03076923076924
